In [ ]:
using PyPlot

nₜ, nₛ = 600, 10 # number of time points, samples

x = linspace(0.,1.,nₜ)
𝐟 = sin.(6.*x).* x.^2

𝐑 = 0.1 ^ 2

𝒊ₓ = sort!(1 + round.(Int, nₜ * rand(nₛ)))

xᵢ = x[𝒊ₓ]

𝐟ᵢ = 𝐟[𝒊ₓ] + chol(𝐑) * randn(nₛ)

plot(x,𝐟,"-",xᵢ,𝐟ᵢ,"o")
xlabel("x"), ylabel("f(x)")
legend(labels=["f(x)","f(xᵢ)"])

In [ ]:
function gp() 
    q, λ = 1., 1.

    k(x,x_) = (q/2λ) * exp(-λ*abs(x - x_))

    𝐊ʸʸ, 𝐊ᶠʸ, 𝐊ᶠᶠ = zeros(nₛ, nₛ), zeros(nₜ, nₛ), zeros(nₜ, nₜ)

    for 𝑖 ∈ 1:nₛ
        for 𝑗 ∈ 1:nₛ
            𝐊ʸʸ[𝑖,𝑗] = k(xᵢ[𝑖],xᵢ[𝑗])
        end
    end

    𝐊ʸʸ += 𝐑 * eye(nₛ)

    for 𝑖 ∈ 1:nₜ
        for 𝑗 ∈ 1:nₛ
            𝐊ᶠʸ[𝑖,𝑗] = k(x[𝑖],xᵢ[𝑗])
        end
    end

    for 𝑖 ∈ 1:nₜ
        for 𝑗 ∈ 1:nₜ
            𝐊ᶠᶠ[𝑖,𝑗] = k(x[𝑖],x[𝑗])
        end
    end

    𝛍 = (𝐊ᶠʸ / 𝐊ʸʸ) * 𝐟ᵢ
    𝚺 = 𝐊ᶠᶠ - (𝐊ᶠʸ / 𝐊ʸʸ) * 𝐊ᶠʸ'
    
    return 𝛍, 𝚺
end


In [ ]:
function plot_estimate(𝛍, 𝚺, 𝛍g=0, 𝚺g=0, txt="full GP")
    figure()
    plot(xᵢ,𝐟ᵢ,"k.")

    𝗖𝗜 = [𝛍 𝛍] + 1.96*[-sqrt.(𝚺) sqrt.(𝚺)]

    plot(x,𝛍,"c--")
    fill_between(x,𝗖𝗜[:,1],𝗖𝗜[:,2],color="c",alpha=0.25)
    
    if 𝛍g != 0
        
        𝗖𝗜 = [𝛍g 𝛍g] + 1.96*[-sqrt.(𝚺g) sqrt.(𝚺g)]
        
        plot(x,𝗖𝗜[:,1],"k--",x,𝗖𝗜[:,2],"k--", alpha=0.5)
        lbls = ["samples", txt, "full GP covariance"]
    else
        lbls = ["samples", txt]
    end
    xlim([0.,1.])
    ylim([-1.5,1.5])
    
    xlabel(L"$t$"), ylabel(L"$x(t)$")
    legend(labels=lbls)
end

In [ ]:
Δₜ = x[2] - x[1]

function kf()
    q, λ = 1., 1.
    
    μ₀,Σ₀ = 0., q/2λ
    
    𝐅 = exp(-Δₜ * λ)
    𝐐 = (q/2λ) * (1 - exp(-2Δₜ * λ))
    
    𝛍 = zeros(nₜ)
    𝚺 = zeros(nₜ)
    
    μ, σ² = μ₀, Σ₀
    𝛍[1],𝚺[1] = μ, σ²
    
    𝑖 = 1
    for t ∈ 2:nₜ
        # predict
        μ, σ² = 𝐅*μ, 𝐅*σ²*𝐅' + 𝐐
        
        if t ∈ 𝒊ₓ
            𝐊ₜ = σ² / (σ² + 𝐑)
            μ  += 𝐊ₜ * (𝐟ᵢ[𝑖] - μ)
            σ² -= 𝐊ₜ * (σ² + 𝐑) * 𝐊ₜ'
            𝑖 += 1
        end
        𝛍[t] = μ
        𝚺[t] = σ²
    end
    
    return 𝛍, 𝚺
end

function rts(𝛍, 𝚺)
    q, λ = 1., 1.
    
    𝐅 = exp(-Δₜ * λ)
    𝐐 = (q/2λ) * (1 - exp(-2Δₜ * λ))
    
  #  𝐆 = []
    
    for t ∈ nₜ-1:-1:1
        
        μ = 𝛍[t]
        σ²= 𝚺[t]
        
        μ⁺, σ²⁺ = 𝐅*μ, 𝐅*σ²*𝐅' + 𝐐
        
        𝐆ₜ = σ² * 𝐅' / σ²⁺
        
        𝛍[t] += 𝐆ₜ * (𝛍[t+1] - μ⁺)
        𝚺[t] += 𝐆ₜ * (𝚺[t+1] - σ²⁺) * 𝐆ₜ'
        
        #𝐆 = vcat(𝐆ₜ,𝐆) 
        
    end
    
    
    return 𝛍, 𝚺#, 𝐆
end

In [ ]:
figure()
plot(xᵢ,𝐟ᵢ,"k.")

xlim([0.,1.])
ylim([-1.5,1.5])

xlabel(L"$t$"), ylabel(L"$x(t)$")

𝛍, 𝚺 = gp()
plot_estimate(𝛍, diag(𝚺))

𝚺f = 𝚺
𝛍g, 𝚺g = 𝛍, diag(𝚺)

𝛍, 𝚺 = kf()
plot_estimate(𝛍, 𝚺, 𝛍g, 𝚺g, "filtering")

𝛍, 𝚺 = rts(𝛍, 𝚺)
plot_estimate(𝛍, 𝚺, 𝛍g, 𝚺g, "smoothing")

𝛍s, 𝚺s = 𝛍, 𝚺

figure()
for i in 1:100
    y = 𝛍g + 𝚺f*randn(length(x),1)
    plot(x,y,"k-",alpha=0.2)
end
xlim([0.,1.])
ylim([-1.5,1.5])

xlabel(L"$t$"), ylabel(L"$x(t)$")


In [ ]:
function construct_posterior(𝚺, 𝐆)
    #K = diagm(𝚺)
    K = zeros(length(𝚺),length(𝚺))
    
    for i = 1:size(K,1)
        for j = 1:i-1
            K[i,j] = prod(𝐆[j:i-1]) * 𝚺[i]
        end
    end
    
    return K + K' + diagm(𝚺)
end

K = construct_posterior(𝚺s, 𝐆)

matshow(𝚺f)
matshow(K)

figure()
for i in 1:100
    y = 𝛍s + K*randn(length(x),1)
    plot(x,y,"k-",alpha=0.2)
end
xlim([0.,1.])
ylim([-1.5,1.5])

xlabel(L"$t$"), ylabel(L"$x(t)$")

In [ ]:
#Pkg.add("BenchmarkTools")

using BenchmarkTools

function kfs()
    𝛍, 𝚺 = kf()
    𝛍, 𝚺 = rts(𝛍, 𝚺)
    return 𝛍, 𝚺
end

function kfs_full()
    𝛍, 𝚺 = kf()
    𝛍, 𝚺, 𝐆 = rts(𝛍, 𝚺)
    𝚺 = construct_posterior(𝚺, 𝐆)
    return 𝛍, 𝚺
end



In [ ]:
@benchmark gp()

In [ ]:
@benchmark kfs()

In [ ]:
@benchmark kfs_full()